# Task 1: Binary Representations

This section implements fundamental bitwise operations used in cryptographic functions, , following the specifications from [FIPS 180-4: Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- **rotl(x, n)**: Left-rotates a 32-bit integer `x` by `n` places.
- **rotr(x, n)**: Right-rotates a 32-bit integer `x` by `n` places.
- **ch(x, y, z)**: Chooses bits based on `x` (from `y` if `x` has 1s, from `z` otherwise).
- **maj(x, y, z)**: Computes the majority vote of the bits from `x, y, z`.

In [21]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the left by n places.
    
    Args:
    - x (int): A 32-bit integer.
    - n (int): Number of positions to rotate.
    
    Returns:
    - int: The rotated integer.
    """
    n = n % 32  # Ensure n is within 0-31
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF  # Mask to 32 bits

def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the right by n places.
    """
    n = n % 32
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
    """
    return (x & y) ^ (~x & z)

def maj(x, y, z):
    """
    Takes a majority vote of the bits in x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)


Testing the Bitwise Functions:
Three test values are used:
- **x = 0xB3333333**
- **y = 0xCCCCCCCC**
- **z = 0xF0F0F0F0**

Rotate `x`, apply `ch(x, y, z)`, and compute `maj(x, y, z)`.


In [22]:
# Test values
test_x = 0b10110011001100110011001100110011  # 0xB3333333
test_y = 0b11001100110011001100110011001100  # 0xCCCCCCCC
test_z = 0b11110000111100001111000011110000  # 0xF0F0F0F0

# Print the results
print(f"rotl({test_x:032b}, 4)  -> {rotl(test_x, 4):032b}")
print(f"rotr({test_x:032b}, 4)  -> {rotr(test_x, 4):032b}")
print(f"ch({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {ch(test_x, test_y, test_z):032b}")
print(f"maj({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {maj(test_x, test_y, test_z):032b}")


rotl(10110011001100110011001100110011, 4)  -> 00110011001100110011001100111011
rotr(10110011001100110011001100110011, 4)  -> 00111011001100110011001100110011
ch(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11000000110000001100000011000000
maj(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11110000111100001111000011110000


# Task 2: Hash Functions

This function computes a simple hash value for a string using the following logic:
1. Initialize `hashval = 0`.
2. Iterate over each character in the string.
3. Compute `hashval = ord(char) + 31 * hashval`.
4. Take the final value modulo 101.

This method ensures good hash distribution and minimizes collisions.


In [23]:
def hash_function(s: str) -> int:
    """
    Computes a simple hash value for a string using a rolling hash approach.
    
    Args:
    - s (str): The input string.
    
    Returns:
    - int: The hash value modulo 101.
    """
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval  # Hash function using 31
    return hashval % 101  # Modulo 101 to limit hash size


Testing the Hash Function:
The function is tested with different strings to observe the hash values.


In [24]:
# Test cases
test_strings = ["hello", "world", "hashing", "kernighan", "ritchie"]

# Compute hashes
for s in test_strings:
    print(f"Hash of '{s}': {hash_function(s)}")


Hash of 'hello': 17
Hash of 'world': 34
Hash of 'hashing': 25
Hash of 'kernighan': 37
Hash of 'ritchie': 26


Use of 31 in Hash Functions:

The number 31 is commonly used in [hash values](https://www.geeksforgeeks.org/why-does-javas-hashcode-in-string-use-31-as-a-multiplier/)., such as Java's String.hashCode(), because it's a prime number, which helps in achieving a uniform distribution of hash values and reduces collisions.


Use of 101 as Modulus in Hash Functions:

Using a prime number like 101 as the [modulus in hash functions](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus) ensures a more uniform distribution of hash values, minimizing clustering and collisions. This practice enhances the efficiency and effectiveness of the hash function.


# Task 3: SHA256 Padding

SHA256 requires messages to be padded to a multiple of 512 bits, as specified in [FIPS 180-4, Section 5.1.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).
This function:
1. Reads a file's contents.
2. Appends a `1` bit (`0x80` in hex).
3. Adds `0` bits until the total length is `56 mod 64`.
4. Appends the original message length as a big-endian 64-bit integer.

This padding ensures compatibility with SHA256 hashing.


In [25]:
import struct
import os

def calculate_sha256_padding(file_path):
    """
    Computes the SHA256 padding for a given file.
    
    Args:
    - file_path (str): The path to the input file.
    
    Prints:
    - The padding bytes in hexadecimal.
    """
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return
    
    with open(file_path, 'rb') as f:
        data = f.read()

    print(f"Read {len(data)} bytes from {file_path}")

    original_length = len(data)
    original_bit_length = original_length * 8  # Convert bytes to bits

    # Append '1' bit (0x80 in hex)
    padding = b'\x80'

    # Compute required zero padding
    total_length = original_length + 1
    while (total_length + 8) % 64 != 0:
        padding += b'\x00'
        total_length += 1

    # Append original length in bits as a big-endian 64-bit integer
    padding += struct.pack('>Q', original_bit_length)

    print(f"Padding length: {len(padding)} bytes")
    print("Padding (Hex):", " ".join(f"{byte:02x}" for byte in padding))


Testing the SHA256 Padding Function:
We apply the padding function to 'test.txt`, which contains a short message.


In [26]:
# Run the padding function
calculate_sha256_padding("test.txt")

Read 3 bytes from test.txt
Padding length: 61 bytes
Padding (Hex): 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


For more details on hash function design:
- [Why does Java's hashCode() use 31 as a multiplier?](https://www.geeksforgeeks.org/why-does-javas-hashcode-in-string-use-31-as-a-multiplier/)
- [Prime Numbers in Hash Functions](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus)
- [Hash Function Design Principles](https://www.cs.hmc.edu/~geoff/classes/hmc.cs070.200101/homework10/hashfuncs.html)

# Task 4: Calculate the First 100 Prime Numbers Using Two Algorithms

## Overview
This notebook computes the first 100 prime numbers using:
1. **Trial Division Method** - A basic but slow approach.
2. **Sieve of Eratosthenes** - A more efficient method for generating primes.


## Implementing the [Trial Division Method](https://www.geeksforgeeks.org/trial-division-algorithm-for-prime-factorization/)

### How It Works:
- A number **n** is prime if it is **only divisible by 1 and itself**.
- To check if a number is prime:
  1. Start from `n = 2` and check divisibility up to `sqrt(n)`.
  2. If no divisor is found, it is prime.
  3. Continue finding primes until we reach **100 primes**.

#### **Efficiency**:
- **Time Complexity**: \( O(n \sqrt{n}) \) (not optimal for large numbers).


In [27]:
import math

def is_prime_trial_division(n):
    """Checks if a number is prime using trial division."""
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def first_n_primes_trial_division(n):
    """Finds the first n prime numbers using the trial division method."""
    primes = []
    num = 2  # Start checking from 2
    while len(primes) < n:
        if is_prime_trial_division(num):
            primes.append(num)
        num += 1
    return primes

# Compute the first 100 primes using Trial Division
primes_trial_division = first_n_primes_trial_division(100)

# Display the first 100 primes
primes_trial_division[:100]


[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541]

##  Implementing the [Sieve of Eratosthenes](https://cp-algorithms.com/algebra/sieve-of-eratosthenes.html)

### How It Works:
1. Create a **boolean array** of size `N`, marking all numbers as **prime**.
2. Start with the smallest prime (`2`) and **mark all multiples as non-prime**.
3. Repeat for the next **unmarked number**.
4. Continue until the required number of primes are reached

#### **Efficiency**:
- **Time Complexity**: \( O(n \log \log n) \) (faster than trial division).
- **Space Complexity**: \( O(n) \) (requires an array to track primes).


In [28]:
def sieve_of_eratosthenes(limit):
    """Finds all prime numbers up to a given limit using the Sieve of Eratosthenes."""
    sieve = [True] * (limit + 1)  # Assume all numbers are prime
    sieve[0] = sieve[1] = False  # 0 and 1 are not prime

    for start in range(2, int(math.sqrt(limit)) + 1):
        if sieve[start]:  # If it's still marked as prime
            for multiple in range(start * start, limit + 1, start):
                sieve[multiple] = False  # Mark multiples as non-prime

    return [num for num, is_prime in enumerate(sieve) if is_prime]

# Approximate limit to get at least 100 primes
primes_sieve = sieve_of_eratosthenes(542)[:100]  # 542 is an estimate

# Display the first 100 primes
primes_sieve[:100]

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541]

# Task 5: Roots
This task involves:
1. **Finding the first 100 prime numbers**.
2. **Computing their square root**.
3. **Extracting the fractional part of the square root**.
4. **Extracting the first 32 bits and converting them to hexadecimal & binary**.


## Find the First 100 Prime Numbers
To perform the calculations, it is first needed to generate **the first 100 prime numbers**. 
We will use the **Trial Division method**, which checks divisibility up to \( \sqrt{n} \).


In [29]:
import math
import numpy as np

def is_prime(n):
    """Checks if a number is prime using trial division."""
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def first_n_primes(n):
    """Finds the first n prime numbers using trial division."""
    primes = []
    num = 2
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

# Compute the first 100 prime numbers
primes = first_n_primes(100)

# Display the first 10 primes for verification
primes[:10]


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

## Extract First 32 Bits of the Fractional Part

For each prime number:
1. Compute its **square root**.
2. Extract the **fractional part** (decimal portion).
3. Multiply by \(2^{32}\) to shift the fractional bits in front of the decimal point.
4. Convert to **binary and hexadecimal formats**.

In [30]:
fractional_bits_32 = []  # Store 32-bit fractional values

for num in primes:  
    # Find the square root of the prime number
    root_value = np.sqrt(num)
    
    # Get only the fractional part (discard the integer part)
    fraction = root_value % 1  

    # Shift left by 32 bits to extract the first 32 bits
    shifted_fraction = fraction * (1 << 32)

    # Convert to an integer
    fraction_bits = int(shifted_fraction)

    # Store the result
    fractional_bits_32.append(fraction_bits)

# Show first 10 values for verification
list(zip(primes[:10], fractional_bits_32[:10]))


[(2, 1779033703),
 (3, 3144134277),
 (5, 1013904242),
 (7, 2773480762),
 (11, 1359893119),
 (13, 2600822924),
 (17, 528734635),
 (19, 1541459225),
 (23, 3418070365),
 (29, 1654270250)]

## Display Results in Hexadecimal and Binary Formats

The format is tabled to show:
- **Prime Number**.
- **First 32 Bits (Hexadecimal)**.
- **First 32 Bits (Binary)**.


# Task 6: Proof of Work

## Objective
In this task, the aim is to:
1. Search through a list of valid English words.
2. Calculate the SHA-256 hash of each word.
3. Count the number of **leading zero bits** in each hash's binary form.
4. Identify the word(s) with the **most leading zero bits**.
5. Prove that each word found is valid by showing it's in a dictionary.

This simulates a simplified version of a **proof-of-work system** used in cryptocurrencies like Bitcoin.


In [31]:
# Read words from the local file `words.txt`
with open("words.txt", "r") as f:
    word_list = {line.strip() for line in f if line.strip()}

print(f"Loaded {len(word_list)} words from words.txt")


Loaded 45373 words from words.txt


In [32]:
import hashlib

def get_sha256_binary(word):
    """Returns 256-bit binary SHA-256 hash of a word."""
    digest = hashlib.sha256(word.encode()).hexdigest()
    return bin(int(digest, 16))[2:].zfill(256)

def count_leading_zeros(binary_str):
    """Counts the number of leading 0 bits in a binary string."""
    return len(binary_str) - len(binary_str.lstrip('0'))


In [33]:
max_zeros = 0
best_words = []

for word in word_list:
    binary_hash = get_sha256_binary(word)
    zeros = count_leading_zeros(binary_hash)
    
    if zeros > max_zeros:
        max_zeros = zeros
        best_words = [(word, binary_hash, zeros)]
    elif zeros == max_zeros:
        best_words.append((word, binary_hash, zeros))


In [34]:
import pandas as pd

df = pd.DataFrame(best_words, columns=["Word", "Binary Hash", "Leading Zero Bits"])
df["Hash (First 64 Bits)"] = df["Binary Hash"].str[:64]
df = df.drop(columns=["Binary Hash"])

display(df)


,Word,Leading Zero Bits,Hash (First 64 Bits)
0,APPLICANT,16,0000000000000000110010100000000110101101110010...


# Task 7: Turing Machines

## Objective
Design a Turing Machine that:
- Starts at the **leftmost bit** of a binary number on the tape.
- Adds **1** to the number, treating it as binary.
- Assumes the **rightmost symbol is the least significant bit (LSB)**.
- Ends with the correct binary result on the tape.

### Example:
- Input on tape: `100111`
- Output after machine halts: `101000`

This Turing Machine performs binary addition by simulating a carry-bit addition from the LSB to the MSB.


## Design Explanation

The Turing Machine will perform the following steps:

1. **Move right** until it reaches the end of the number (the rightmost bit).
2. **Perform addition** from right to left:
   - If it sees a `0`, change it to `1` and **halt**.
   - If it sees a `1`, change it to `0` and **carry the 1** to the left.
   - If it reaches a blank (meaning all bits were `1`), **insert `1` at the front**.
3. The machine halts when the addition is complete.


##  States and Transitions

| State     | Read | Write | Move | Next State |
|-----------|------|-------|------|------------|
| `q0`      | 0/1  | 0/1   | R    | `q0`       |
| `q0`      | □    | □     | L    | `q1`       |
| `q1`      | 0    | 1     | N    | `HALT`     |
| `q1`      | 1    | 0     | L    | `q1`       |
| `q1`      | □    | 1     | N    | `HALT`     |

**State Descriptions:**
- `q0`: Move to the end of the binary number.
- `q1`: Move back left, applying the addition.


In [35]:
def binary_add_one_turing_machine(tape_str):
    tape = list(tape_str) + ['□']  # Add blank at the end
    head = 0
    state = 'q0'

    while True:
        symbol = tape[head]

        if state == 'q0':
            if symbol in ('0', '1'):
                head += 1  # Move right
            elif symbol == '□':
                head -= 1  # Move left
                state = 'q1'

        elif state == 'q1':
            if tape[head] == '0':
                tape[head] = '1'
                break  # Done, halt
            elif tape[head] == '1':
                tape[head] = '0'
                head -= 1  # Carry the 1
            elif tape[head] == '□':
                tape[head] = '1'  # All 1's (e.g. 111 + 1 = 1000)
                break

    return ''.join(tape).rstrip('□')  # Remove trailing blank

# Example test
binary_add_one_turing_machine("100111")


'101000'

## Output

Input: `100111`  
Output: `101000`

This matches the expected result.


# Task 8: Computational Complexity

## Objective
- Implement the **Bubble Sort** algorithm in Python.
- Modify it to **count the number of comparisons** made during sorting.
- Apply it to **all permutations** of the list `L = [1, 2, 3, 4, 5]`.
- For each permutation:
  - Print the permutation.
  - Print the number of comparisons Bubble Sort made to sort it.

In [36]:
import itertools

def bubble_sort_with_comparisons(arr):
    """
    Bubble sort implementation that counts comparisons.
    
    Args:
    - arr (list): The input list to sort
    
    Returns:
    - sorted_list (list): The sorted version of the input
    - comparison_count (int): The number of element comparisons made
    """
    a = arr.copy()  # Make a copy to avoid modifying the original
    n = len(a)
    comparisons = 0

    for i in range(n):
        for j in range(0, n - i - 1):
            comparisons += 1  # Count this comparison
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
    
    return a, comparisons


## Generate All Permutations of [1, 2, 3, 4, 5]

Generate every possible ordering (5! = 120 permutations).


In [37]:
L = [1, 2, 3, 4, 5]
permutations = list(itertools.permutations(L))


## Run Bubble Sort on All Permutations

- Sort each permutation.
- Count the number of comparisons.
- Print both the permutation and the comparison count.


In [38]:
def new_func(perm):
    _, comparisons = bubble_sort_with_comparisons(list(perm))
    return comparisons

for perm in permutations:
    comparisons = new_func(perm)
    print(f"Permutation: {perm} -> Comparisons: {comparisons}")


Permutation: (1, 2, 3, 4, 5) -> Comparisons: 10
Permutation: (1, 2, 3, 5, 4) -> Comparisons: 10
Permutation: (1, 2, 4, 3, 5) -> Comparisons: 10
Permutation: (1, 2, 4, 5, 3) -> Comparisons: 10
Permutation: (1, 2, 5, 3, 4) -> Comparisons: 10
Permutation: (1, 2, 5, 4, 3) -> Comparisons: 10
Permutation: (1, 3, 2, 4, 5) -> Comparisons: 10
Permutation: (1, 3, 2, 5, 4) -> Comparisons: 10
Permutation: (1, 3, 4, 2, 5) -> Comparisons: 10
Permutation: (1, 3, 4, 5, 2) -> Comparisons: 10
Permutation: (1, 3, 5, 2, 4) -> Comparisons: 10
Permutation: (1, 3, 5, 4, 2) -> Comparisons: 10
Permutation: (1, 4, 2, 3, 5) -> Comparisons: 10
Permutation: (1, 4, 2, 5, 3) -> Comparisons: 10
Permutation: (1, 4, 3, 2, 5) -> Comparisons: 10
Permutation: (1, 4, 3, 5, 2) -> Comparisons: 10
Permutation: (1, 4, 5, 2, 3) -> Comparisons: 10
Permutation: (1, 4, 5, 3, 2) -> Comparisons: 10
Permutation: (1, 5, 2, 3, 4) -> Comparisons: 10
Permutation: (1, 5, 2, 4, 3) -> Comparisons: 10
Permutation: (1, 5, 3, 2, 4) -> Comparis